In [1]:
import tensorflow as tf

In [2]:
import shapely
import geopandas as gpd
from shapely.geometry import box

In [3]:
from osgeo import gdal, ogr

In [4]:
import glob
import os

In [5]:
rasters = glob.glob('C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip/*.jp2')
print(rasters)

['C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\\top15-nc-cir-50cm-baylor_48023.jp2', 'C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\\top15-nc-cir-50cm-camp_48063.jp2', 'C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\\top15-nc-cir-50cm-elpaso_48141.jp2', 'C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\\top15-nc-cir-50cm-galveston_48167.jp2', 'C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\\top15-nc-cir-50cm-travis_48453.jp2']


In [6]:
shps = glob.glob('C:/Users/joelw/Python3_Vehicle_ATR/nonvehicle_polygons/*.shp')
print(shps)

['C:/Users/joelw/Python3_Vehicle_ATR/nonvehicle_polygons\\nonvehicles_galveston.shp']


I think the best way to do this is to open a raster. while it's open pull in the shp cycle through each of the features and see if it is contained within the raster. if it is we will run a gdalwarp on it to extract that section of the raster. we will save the raster into a new folder called image_chips wiht a _n behind the image name.

In [7]:
count = 0
#filter through each raster
for raster in rasters:
    
    #open the raster in gdal
    ds = gdal.Open(raster)
    
    
    # Get raster geometry
    transform = ds.GetGeoTransform()
    pixelWidth = transform[1]
    pixelHeight = transform[5]
    cols = ds.RasterXSize
    rows = ds.RasterYSize

    xLeft = transform[0]
    yTop = transform[3]
    xRight = xLeft+cols*pixelWidth
    yBottom = yTop-rows*pixelWidth

    ring = ogr.Geometry(ogr.wkbLinearRing)
    ring.AddPoint(xLeft, yTop)
    ring.AddPoint(xLeft, yBottom)
    ring.AddPoint(xRight, yTop)
    ring.AddPoint(xRight, yBottom)
    ring.AddPoint(xLeft, yTop)
    rasterGeometry = ogr.Geometry(ogr.wkbPolygon)
    rasterGeometry.AddGeometry(ring)
    #print("raster name: ", raster)
    #print("raster geometry: ", rasterGeometry)
    
    #open the shapefile in gdal
    for shp in shps:
        
        #shp = r"C:\Users\joelw\Python3_Vehicle_ATR\vehicle_locations_polygons2.shp"
        #print(shp)
        driver = ogr.GetDriverByName('ESRI Shapefile')
        source = ogr.Open(shp)
        layer = source.GetLayer(0)
        featureCount = layer.GetFeatureCount()
        #print(featureCount)
        
        for feature in layer:
            #print("feature ", count, ": ", feature.GetGeometryRef())
            geometry = feature.GetGeometryRef()
            xmin, xmax, ymin, ymax = geometry.GetEnvelope()
            
            #print(geometry.Intersect(rasterGeometry))

            if geometry.Within(rasterGeometry):
                print(count)
                print(raster)
                print(rasterGeometry)
                print(shp)
                print(geometry)
                image_name = "C:/Users/joelw/Python3_Vehicle_ATR/nonvehicle_chips/clipped_image_" + str(count) + ".tif"
                options = gdal.WarpOptions(format = 'GTiff', 
                                           cutlineLayer= geometry, 
                                           cropToCutline=True, 
                                           multithread = True,
                                          outputBounds = (xmin, ymin, xmax, ymax),
                                          srcAlpha = False)
                gdal.Warp(image_name, raster, options = options)
                
                
                count+=1
                #gdal.warp()
                


    

0
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_48167.jp2
POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216400.5 0,281456 3279509 0))
C:/Users/joelw/Python3_Vehicle_ATR/nonvehicle_polygons\nonvehicles_galveston.shp
POLYGON ((313784.905804451 3249644.17298404,313784.905804451 3249632.17298404,313772.905804451 3249632.17298404,313772.905804451 3249644.17298404,313784.905804451 3249644.17298404))
1
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_48167.jp2
POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216400.5 0,281456 3279509 0))
C:/Users/joelw/Python3_Vehicle_ATR/nonvehicle_polygons\nonvehicles_galveston.shp
POLYGON ((313818.746837187 3249568.36907071,313818.746837187 3249556.36907071,313806.746837187 3249556.36907071,313806.746837187 3249568.36907071,313818.746837187 3249568.36907071))
2
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_4816

18
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_48167.jp2
POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216400.5 0,281456 3279509 0))
C:/Users/joelw/Python3_Vehicle_ATR/nonvehicle_polygons\nonvehicles_galveston.shp
POLYGON ((312090.485295362 3248450.26134912,312090.485295362 3248438.26134912,312078.485295362 3248438.26134912,312078.485295362 3248450.26134912,312090.485295362 3248450.26134912))
19
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_48167.jp2
POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216400.5 0,281456 3279509 0))
C:/Users/joelw/Python3_Vehicle_ATR/nonvehicle_polygons\nonvehicles_galveston.shp
POLYGON ((312619.307833587 3249423.07823684,312619.307833587 3249411.07823684,312607.307833587 3249411.07823684,312607.307833587 3249423.07823684,312619.307833587 3249423.07823684))
20
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_4

36
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_48167.jp2
POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216400.5 0,281456 3279509 0))
C:/Users/joelw/Python3_Vehicle_ATR/nonvehicle_polygons\nonvehicles_galveston.shp
POLYGON ((357415.359687179 3267262.2658403,357415.359687179 3267250.2658403,357403.359687179 3267250.2658403,357403.359687179 3267262.2658403,357415.359687179 3267262.2658403))


galveston - zone 15n
camp - zone 15n
el paso - zone 13n
baylor - zone 14n
travis - zone 14n

In [35]:
del(raster)

In [9]:
count = 0
#filter through each raster
for raster in rasters:
    
    #open the raster in gdal
    ds = gdal.Open(raster)
    
    
    # Get raster geometry
    transform = ds.GetGeoTransform()
    pixelWidth = transform[1]
    pixelHeight = transform[5]
    cols = ds.RasterXSize
    rows = ds.RasterYSize

    xLeft = transform[0]
    yTop = transform[3]
    xRight = xLeft+cols*pixelWidth
    yBottom = yTop-rows*pixelWidth

    ring = ogr.Geometry(ogr.wkbLinearRing)
    ring.AddPoint(xLeft, yTop)
    ring.AddPoint(xLeft, yBottom)
    ring.AddPoint(xRight, yTop)
    ring.AddPoint(xRight, yBottom)
    ring.AddPoint(xLeft, yTop)
    rasterGeometry = ogr.Geometry(ogr.wkbPolygon)
    rasterGeometry.AddGeometry(ring)
    #print("raster name: ", raster)
    #print("raster geometry: ", rasterGeometry)
    
    #open the shapefile in gdal
    for shp in shps:
        
        #shp = r"C:\Users\joelw\Python3_Vehicle_ATR\vehicle_locations_polygons2.shp"
        #print(shp)
        driver = ogr.GetDriverByName('ESRI Shapefile')
        source = ogr.Open(shp)
        layer = source.GetLayer(0)
        featureCount = layer.GetFeatureCount()
        #print(featureCount)
        
        for feature in layer:
            #print("feature ", count, ": ", feature.GetGeometryRef())
            geometry = feature.GetGeometryRef()
            xmin, xmax, ymin, ymax = geometry.GetEnvelope()
            
            #print(geometry.Intersect(rasterGeometry))

            if geometry.Within(rasterGeometry):
                print(count)
                print(raster)
                print(rasterGeometry)
                print(shp)
                print(geometry)
                image_name = "C:/Users/joelw/Python3_Vehicle_ATR/image_chips/clipped_image_" + str(count) + ".png"
                options = gdal.WarpOptions(format = 'PNG', 
                                           cutlineLayer= geometry, 
                                           cropToCutline=True, 
                                           multithread = True,
                                          outputBounds = (xmin, ymin, xmax, ymax),
                                          srcAlpha = False)
                gdal.Warp(image_name, raster, options = options)
                
                
                count+=1
                #gdal.warp()
                


    

0
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-baylor_48023.jp2
POLYGON ((453330 3748569 0,453330 3692700 0,505974.5 3748569.0 0,505974.5 3692700.0 0,453330 3748569 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_travis_baylor14.shp
POLYGON ((473093.269197676 3717380.14929816,473093.269197676 3717368.14929816,473081.269197676 3717368.14929816,473081.269197676 3717380.14929816,473093.269197676 3717380.14929816))
1
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-baylor_48023.jp2
POLYGON ((453330 3748569 0,453330 3692700 0,505974.5 3748569.0 0,505974.5 3692700.0 0,453330 3748569 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_travis_baylor14.shp
POLYGON ((472599.045637329 3717382.30296541,472599.045637329 3717370.30296541,472587.045637329 3717370.30296541,472587.045637329 3717382.30296541,472599.045637329 3717382.30296541))
2
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-baylor_48023.jp2
POL

POLYGON ((453330 3748569 0,453330 3692700 0,505974.5 3748569.0 0,505974.5 3692700.0 0,453330 3748569 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_travis_baylor14.shp
POLYGON ((475329.357292507 3716272.21671848,475329.357292507 3716260.21671848,475317.357292507 3716260.21671848,475317.357292507 3716272.21671848,475329.357292507 3716272.21671848))
25
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-baylor_48023.jp2
POLYGON ((453330 3748569 0,453330 3692700 0,505974.5 3748569.0 0,505974.5 3692700.0 0,453330 3748569 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_travis_baylor14.shp
POLYGON ((475270.605249949 3716224.31915885,475270.605249949 3716212.31915885,475258.605249949 3716212.31915885,475258.605249949 3716224.31915885,475270.605249949 3716224.31915885))
26
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-baylor_48023.jp2
POLYGON ((453330 3748569 0,453330 3692700 0,505974.5 3748569.0 0,505974.5 3692700.0 0,45

POLYGON ((453330 3748569 0,453330 3692700 0,505974.5 3748569.0 0,505974.5 3692700.0 0,453330 3748569 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_travis_baylor14.shp
POLYGON ((475529.131466543 3716634.20510972,475529.131466543 3716622.20510972,475517.131466543 3716622.20510972,475517.131466543 3716634.20510972,475529.131466543 3716634.20510972))
49
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-baylor_48023.jp2
POLYGON ((453330 3748569 0,453330 3692700 0,505974.5 3748569.0 0,505974.5 3692700.0 0,453330 3748569 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_travis_baylor14.shp
POLYGON ((475505.268833422 3716641.35528499,475505.268833422 3716629.35528499,475493.268833422 3716629.35528499,475493.268833422 3716641.35528499,475505.268833422 3716641.35528499))
50
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-baylor_48023.jp2
POLYGON ((453330 3748569 0,453330 3692700 0,505974.5 3748569.0 0,505974.5 3692700.0 0,45

POLYGON ((453330 3748569 0,453330 3692700 0,505974.5 3748569.0 0,505974.5 3692700.0 0,453330 3748569 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_travis_baylor14.shp
POLYGON ((475834.779922551 3717406.33789191,475834.779922551 3717394.33789191,475822.779922551 3717394.33789191,475822.779922551 3717406.33789191,475834.779922551 3717406.33789191))
72
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-baylor_48023.jp2
POLYGON ((453330 3748569 0,453330 3692700 0,505974.5 3748569.0 0,505974.5 3692700.0 0,453330 3748569 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_travis_baylor14.shp
POLYGON ((475846.582019076 3717584.66154014,475846.582019076 3717572.66154014,475834.582019076 3717572.66154014,475834.582019076 3717584.66154014,475846.582019076 3717584.66154014))
73
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-baylor_48023.jp2
POLYGON ((453330 3748569 0,453330 3692700 0,505974.5 3748569.0 0,505974.5 3692700.0 0,45

POLYGON ((295184.5 3667198.0 0,295184.5 3638531.0 0,342401 3667198 0,342401 3638531 0,295184.5 3667198.0 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_galveston_camp15.shp
POLYGON ((316181.158489768 3652334.81513229,316181.158489768 3652322.81513229,316169.158489768 3652322.81513229,316169.158489768 3652334.81513229,316181.158489768 3652334.81513229))
97
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-camp_48063.jp2
POLYGON ((295184.5 3667198.0 0,295184.5 3638531.0 0,342401 3667198 0,342401 3638531 0,295184.5 3667198.0 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_galveston_camp15.shp
POLYGON ((316230.869939818 3652395.62313959,316230.869939818 3652383.62313959,316218.869939818 3652383.62313959,316218.869939818 3652395.62313959,316230.869939818 3652395.62313959))
98
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-camp_48063.jp2
POLYGON ((295184.5 3667198.0 0,295184.5 3638531.0 0,342401 3667198 0,342401 363853

...

POLYGON ((340112.5 3548678.5 0,340112.5 3471381.5 0,411667.0 3548678.5 0,411667.0 3471381.5 0,340112.5 3548678.5 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_elpaso13.shp
POLYGON ((358789.673226682 3514195.76567826,358789.673226682 3514183.76567826,358777.673226682 3514183.76567826,358777.673226682 3514195.76567826,358789.673226682 3514195.76567826))
322
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-elpaso_48141.jp2
POLYGON ((340112.5 3548678.5 0,340112.5 3471381.5 0,411667.0 3548678.5 0,411667.0 3471381.5 0,340112.5 3548678.5 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_elpaso13.shp
POLYGON ((358837.994984023 3514216.87016558,358837.994984023 3514204.87016558,358825.994984023 3514204.87016558,358825.994984023 3514216.87016558,358837.994984023 3514216.87016558))
323
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-elpaso_48141.jp2
POLYGON ((340112.5 3548678.5 0,340112.5 3471381.5 0,411667.0 3548678.5 0,411

POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216400.5 0,281456 3279509 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_galveston_camp15.shp
POLYGON ((308899.792667301 3249422.44225602,308899.792667301 3249410.44225602,308887.792667301 3249410.44225602,308887.792667301 3249422.44225602,308899.792667301 3249422.44225602))
356
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_48167.jp2
POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216400.5 0,281456 3279509 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_galveston_camp15.shp
POLYGON ((308614.983725297 3249294.85861899,308614.983725297 3249282.85861899,308602.983725297 3249282.85861899,308602.983725297 3249294.85861899,308614.983725297 3249294.85861899))
357
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_48167.jp2
POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216

POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216400.5 0,281456 3279509 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_galveston_camp15.shp
POLYGON ((308056.769762071 3250176.24138956,308056.769762071 3250164.24138956,308044.769762071 3250164.24138956,308044.769762071 3250176.24138956,308056.769762071 3250176.24138956))
375
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_48167.jp2
POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216400.5 0,281456 3279509 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_galveston_camp15.shp
POLYGON ((308620.772937492 3250178.97781783,308620.772937492 3250166.97781783,308608.772937492 3250166.97781783,308608.772937492 3250178.97781783,308620.772937492 3250178.97781783))
376
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_48167.jp2
POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216

POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216400.5 0,281456 3279509 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_galveston_camp15.shp
POLYGON ((306849.890683487 3251610.78281922,306849.890683487 3251598.78281922,306837.890683487 3251598.78281922,306837.890683487 3251610.78281922,306849.890683487 3251610.78281922))
394
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_48167.jp2
POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216400.5 0,281456 3279509 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_galveston_camp15.shp
POLYGON ((306892.769289946 3251648.84106406,306892.769289946 3251636.84106406,306880.769289946 3251636.84106406,306880.769289946 3251648.84106406,306892.769289946 3251648.84106406))
395
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_48167.jp2
POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216

POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216400.5 0,281456 3279509 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_galveston_camp15.shp
POLYGON ((289820.690606051 3268495.92089381,289820.690606051 3268483.92089381,289808.690606051 3268483.92089381,289808.690606051 3268495.92089381,289820.690606051 3268495.92089381))
414
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_48167.jp2
POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216400.5 0,281456 3279509 0))
C:/Users/joelw/Python3_Vehicle_ATR/vehicle_polygons\vehicles_galveston_camp15.shp
POLYGON ((289756.626074579 3268634.30454562,289756.626074579 3268622.30454562,289744.626074579 3268622.30454562,289744.626074579 3268634.30454562,289756.626074579 3268634.30454562))
415
C:/Users/joelw/Python3_Vehicle_ATR/images_to_clip\top15-nc-cir-50cm-galveston_48167.jp2
POLYGON ((281456 3279509 0,281456.0 3216400.5 0,373094 3279509 0,373094.0 3216

In [16]:
ds

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000027413744390> >

In [17]:
ds.GetRasterBand(1)

<osgeo.gdal.Band; proxy of <Swig Object of type 'GDALRasterBandShadow *' at 0x0000027413B740F0> >

Was able to push the tif images into gdal_translate and gdalwarp to get png images that were readable into cv2.  woot woot.

In [34]:
#for /r image_chips %f in (*.tif) do gdal_translate -of VRT -b 1 -b 2 -b 3 -mask "none" -tr 0.5 0.5 %f image_chips\%~nf.vrt"

In [35]:
#for /r image_chips/ %f in (*.vrt) do gdalwarp -nosrcalpha -of PNG %f image_chips2\%~nf.png